In [2]:
!pip install selenium

### **구현 계획**
<br>
1. 사용자로부터 가격을 알고자 하는 상품의 이름을 입력받고, 이로부터 각 쇼핑 사이트 검색 결과에서 상품 상세 페이지로 들어가 상품명, 상품 가격(구매 옵션 있으면 구매 옵션별 가격), 묶음배송 상품정보, 상품 상세페이지 URL을 추출함 (이때, 쇼핑몰 사이트 별로 추출하는 코드를 각각 구현함) 
<br>
<br>
2. 이 중, 상품명, 상품 가격, 상품 상세 페이지 URL을 csv 형태로 저장 후 사용자에게 보여줌(이때, 상품 가격이 구매 옵션별로 다른 경우 각각의 상품명을 상품명 + 구매 옵션으로 저장 후 각각의 상품 가격을 저장)
<br>
<br>
3. 사용자가 검색을 통해 얻은 상품들 중 특정 한 제품만 구매하고 싶은 경우, 그 상품에 대한 정보만 나타낼수 있도록 필터링 함
<br>
<br>
4. 1. 2. 3.의 과정을 통해서 구매할 상품에 대한 정보를 얻은 후, 구매할 상품이 2개 이상인 경우 묶음배송 상품으로 구매 가능한지 여부를 알아보기 위해 각각의 상품의 묶음배송 상품정보를 2.에서 사용한 방법과 마찬가지로 상품명, 상품 가격, 상품 상세페이지 URL을 csv파일로 저장 후 구매하려하는 다른 상품명으로 검색하여 일치하는 상품이 있는지 확인
<br>
<br>
5. 묶음배송 상품까지 확인한 후, 구매하려는 상품들을 최저가로 구매할 수 있는 조합을 5개까지 추천(기본값, 사용자 설정에 의해 추천하는 구매 조합의 개수를 늘릴 수 있음). 추천하는 조합의 개수를 여러개로 설정하는 이유는 가격 외에 구매를 결정하는 다른 조건(리뷰, 구매자 수 등)을 고려하기 위함.

In [2]:
import requests
import re
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

In [5]:
path = "C:/Users/82106/Downloads/chromedriver.exe"
driver = webdriver.Chrome(path)
driver.get("https://shopping.naver.com/")

search_box = driver.find_element_by_name("query")

item_name = input(prompt = "찾을 품목 이름 입력 : ")
search_box.send_keys(item_name)
search_box.send_keys(Keys.RETURN)

SCROLL_PAUSE_SEC = 0.5

# 스크롤 높이 가져옴
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # 끝까지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # 1초 대기
    time.sleep(SCROLL_PAUSE_SEC)

    # 스크롤 다운 후 스크롤 높이 다시 가져옴
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

찾을 품목 이름 입력 : 파스타


In [7]:
req = driver.page_source
soup = BeautifulSoup(req, "lxml")

item_names = soup.find_all("li", attrs={"class":"basicList_item__2XT81"})
for item in item_names:
    if(item.find("button", attrs={"class":"ad_ad_stk__12U34"})):
        continue
    print(item.find("a", attrs={"class":"basicList_link__1MaTN"}).get_text())
    image = item.find('img')
    
    if(item.find("span", attrs={"class":"price_low__2vp2A"})):
        print(item.find("span", attrs={"class":"price_low__2vp2A"}).get_text())
        print(item.find("span", attrs={"class":"price_num__2WUXn"}).get_text())
    else:
        print(item.find("span", attrs={"class":"price_num__2WUXn"}).get_text())

대한제분 곰표 파스타면 500g 모음전 보가사리 라폰테 스파게티 마카로니 업소용 대용량
740원
투움바 파스타
최저
9,400원
[이가자연면] 투움바 파스타 6개입 박스
9,980원
글루텐프리 파스타면(그라노로) 400g 쌀파스타 분유 아기 비건 스파게티 돌아기
4,800원
유기농 통밀 푸실리 500g 푸질리 분유파스타 펜네 그라노로 아기파스타 통곡물 돌아기
4,000원
유기농 통밀 파스타면 500g
3,800원
청정원 에그페투치니 파스타면
최저
1,480원
디벨라 링귀니 500g / 링기니 14번 500g / 파스타면
1,150원
라리 스파게티 500g 파스타 마카로니
최저
630원
이가자연면 이가 투움바파스타 179.5gx6봉
최저
11,870원
글루텐 프리 쌀파스타, 아로스 쌀파스타
5,000원
프레시지 트러플 크림 파스타 2인분
최저
9,180원
프레시지 바질크림 빠네 파스타
최저
8,547원
라리 삼색 후실리 500g 푸실리 파스타
최저
890원
오뚜기 펜네 파스타
최저
1,280원
전문점용 반조리 생 페투치니 파스타면 (25인분)
14,960원
샘표 폰타나 파스타면 2종
최저
1,750원
스파게티면 업소 스파게티 오뚜기 500g X2개 파스타
최저
800원
유기농 통밀 파스타면 스파게티
최저
3,770원
가능상품 데체코 DECECCO 시금치맛 페투치네 파스타 250g
최저
2,980원
달라코스타 삼색 동물원 공룡 여행 씨라이프 동물 모양 알파벳 펠리체티 유기농 아기 파스타
2,990원
대상 청정원 에그 파스타 탈리올리니 250g
최저
1,650원
데체코 카펠리니 500g 카팰리니 9호 얇은 파스타면
2,700원
프레시지 삼시세끼 명란 파스타 2인분
최저
9,090원
프레시지 쉬림프로제파스타 2인분 9 900원
최저
8,547원
바릴라 스파게티 5호 no.5 1kg 파스타 농심
최저
2,040원
판타넬라 파르팔레 500g 파르펠레 리본 파스타
최저
1,240원
CJ제일제당 백설 파스타 면
최저
2,190원
스파게티면 마카로니 이태리 파스타면
최저
73